In [1]:
import sys
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
from lxml import html as htmlRenderer
import requests
import json
from datetime import date

In [2]:
allUrl = []

class WebPage(QtWebEngineWidgets.QWebEnginePage):
    def __init__(self):
        super(WebPage, self).__init__()
        self._newsAndComments = []
        self._baseUrl = ""
        self._firstPageProcessed = False;
        self.loadFinished.connect(self.handleLoadFinished)

    def start(self, baseUrl):
        self._baseUrl = baseUrl
        self.load(QtCore.QUrl(baseUrl))

    def fetchNext(self):
        try:
            print(" - fetch next url")
            url = next(self._urls)
        except StopIteration:
            return False
        else:
            self.load(QtCore.QUrl(url))
        return True

    def processCurrentPage(self, html):
        url = self.url().toString()
        renderedPage = htmlRenderer.fromstring(html)
        print(" -> processing base URL: {}".format(url))
        auxLinks = renderedPage.xpath("//a/@href")
        print(len(auxLinks))
        print("==================================================================================================")
        allUrl = auxLinks
        QtWidgets.qApp.quit()

    def handleLoadFinished(self):
        self.toHtml(self.processCurrentPage)


In [3]:

url = "https://www.abc.es"
app = QtWidgets.QApplication(sys.argv)
webpage = WebPage()
webpage.start(url)
sys.exit(app.exec_())

 -> processing base URL: https://www.abc.es/
698


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


In [4]:
len(allUrl)

0